<a href="https://colab.research.google.com/github/MR-Toufigh/Basics-of-intelligent-systems-fall-2024/blob/main/The_second_mini_project_Q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns

# Load and preprocess the California Housing dataset
data = fetch_california_housing()
X, y = data.data, data.target

# Normalize the input features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model 1: Neural Network with RBF layer
class RBFLayer(tf.keras.layers.Layer):
    def __init__(self, num_units, gamma):
        super(RBFLayer, self).__init__()
        self.num_units = num_units
        self.gamma = gamma

    def build(self, input_shape):
        self.centers = self.add_weight(name='centers',
                                       shape=(self.num_units, input_shape[-1]),
                                       initializer='uniform',
                                       trainable=True)
        self.betas = self.add_weight(name='betas',
                                     shape=(self.num_units,),
                                     initializer='ones',
                                     trainable=True)

    def call(self, inputs):
        diff = tf.expand_dims(inputs, axis=1) - self.centers
        l2 = tf.reduce_sum(tf.square(diff), axis=-1)
        return tf.exp(-self.gamma * l2)

# Create the RBF model
rbf_model = Sequential([
    Input(shape=(X.shape[1],)),
    RBFLayer(num_units=128, gamma=0.1),
    Dense(1)
])

rbf_model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mse'])
history_rbf = rbf_model.fit(X_train, y_train, epochs=500, batch_size=32, verbose=0, validation_split=0.2)

# Evaluate RBF model
rbf_predictions = rbf_model.predict(X_test)
rbf_loss = mean_squared_error(y_test, rbf_predictions)
rbf_r2 = r2_score(y_test, rbf_predictions)

print(f"RBF Model Loss: {rbf_loss}")
print(f"RBF Model R^2 Score: {rbf_r2}")

# Model 2: Fully Connected Dense Neural Network
dense_model = Sequential([
    Input(shape=(X.shape[1],)),
    Dense(128, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

dense_model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mse'])
history_dense = dense_model.fit(X_train, y_train, epochs=500, batch_size=32, verbose=0, validation_split=0.2)

# Evaluate Dense model
dense_predictions = dense_model.predict(X_test)
dense_loss = mean_squared_error(y_test, dense_predictions)
dense_r2 = r2_score(y_test, dense_predictions)

print(f"Dense Model Loss: {dense_loss}")
print(f"Dense Model R^2 Score: {dense_r2}")

# Create and display confusion matrix for both models
# Note: We convert regression output to categorical by thresholding (e.g., above/below median)
median_value = np.median(y_test)
rbf_classes = (rbf_predictions.flatten() >= median_value).astype(int)
dense_classes = (dense_predictions.flatten() >= median_value).astype(int)
y_test_classes = (y_test >= median_value).astype(int)

rbf_confusion_matrix = confusion_matrix(y_test_classes, rbf_classes)
dense_confusion_matrix = confusion_matrix(y_test_classes, dense_classes)

# Plot confusion matrices as heatmaps
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
sns.heatmap(rbf_confusion_matrix, annot=True, fmt='d', cmap='Blues', ax=axes[0])
axes[0].set_title("RBF Model Confusion Matrix")
axes[0].set_xlabel("Predicted")
axes[0].set_ylabel("Actual")

sns.heatmap(dense_confusion_matrix, annot=True, fmt='d', cmap='Blues', ax=axes[1])
axes[1].set_title("Dense Model Confusion Matrix")
axes[1].set_xlabel("Predicted")
axes[1].set_ylabel("Actual")

plt.tight_layout()
plt.show()

# Plot training and validation loss for both models
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(history_rbf.history['val_mse'], label='RBF Model Validation MSE', linestyle='--')
ax.plot(history_rbf.history['mse'], label='RBF Model Training MSE')
ax.plot(history_dense.history['val_mse'], label='Dense Model Validation MSE', linestyle='--')
ax.plot(history_dense.history['mse'], label='Dense Model Training MSE')
ax.set_title("Training and Validation MSE")
ax.set_xlabel("Epochs")
ax.set_ylabel("Mean Squared Error")
ax.legend()
plt.show()


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:678: UserWarning: Gradients do not exist for variables ['betas'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(
